
# 10.2 TensorFlowの高水準APIを使ったMulti-Layer Perceptron


In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)


1.13.1


In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28)
X_test = X_test.astype(np.float32).reshape(-1, 28*28)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)


In [4]:
X_train.shape


(60000, 784)

In [5]:
X_test.shape


(10000, 784)

In [6]:
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]


In [7]:
X_train.shape


(55000, 784)

In [8]:
feature_cols = [tf.feature_column.numeric_column("X", shape=[28*28])]
dnn_clf = tf.estimator.DNNClassifier(hidden_units=[300,100], n_classes=10, feature_columns=feature_cols)
input_fn = tf.estimator.inputs.numpy_input_fn(x = {"X": X_train}, y = y_train, num_epochs=40, batch_size=50, shuffle=True)
dnn_clf.train(input_fn=input_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/3f/yvs6f9j94nzgm0ywkq35v1yw0000gn/T/tmp4s5_4sra', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12372fc50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
T

In [9]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(x = {"X": X_test}, y = y_test, shuffle=False)
eval_results = dnn_clf.evaluate(input_fn=test_input_fn)
eval_results 


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-02T14:09:31Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/folders/3f/yvs6f9j94nzgm0ywkq35v1yw0000gn/T/tmp4s5_4sra/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-02-14:09:32
INFO:tensorflow:Saving dict for global step 44000: accuracy = 0.9425, average_loss = 0.28849906, global_step = 44000, loss = 36.518867
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 44000: /var/folders/3f/yvs6f9j94nzgm0ywkq35v1yw0000gn/T/tmp4s5_4sra/model.ckpt-44000


{'accuracy': 0.9425,
 'average_loss': 0.28849906,
 'loss': 36.518867,
 'global_step': 44000}

In [10]:
y_pred_iter = dnn_clf.predict(input_fn=test_input_fn)
y_pred = list(y_pred_iter)
y_pred[0]


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/3f/yvs6f9j94nzgm0ywkq35v1yw0000gn/T/tmp4s5_4sra/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'logits': array([ 0.98958987, -1.6613775 , -0.13518932,  0.1280905 ,  0.5633618 ,
        -0.47853243, -1.4908048 ,  7.17769   ,  0.11979096,  2.8638117 ],
       dtype=float32),
 'probabilities': array([2.0135310e-03, 1.4212089e-04, 6.5384217e-04, 8.5077353e-04,
        1.3147702e-03, 4.6383237e-04, 1.6855302e-04, 9.8042905e-01,
        8.4374187e-04, 1.3119664e-02], dtype=float32),
 'class_ids': array([7]),
 'classes': array([b'7'], dtype=object)}


このようにTensorFlowの高水準APIを使うことで非常に簡単にDeep Neural Networkを構築することができる。



## 10.3 プレーンなTensorFlowを使ったDeep Neural Networkの訓練


In [34]:
# ハイパーパラメータ
n_inputs = 28*28 # 画像データ。28*28pixelで1pixelごとに特徴量として扱う
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10 # 文字0-9


In [35]:
# ここから構築フェーズ
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")


In [36]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z
        

In [75]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")


In [68]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")


-----xentropy----- Tensor("loss_4/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits:0", shape=(?,), dtype=float32)
-----loss----- Tensor("loss_4/loss:0", shape=(), dtype=float32)


In [39]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)


In [40]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))


In [41]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
from log import log_dir
file_writer = tf.summary.FileWriter(log_dir("mnist_dnn"), tf.get_default_graph())


In [42]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch



In [69]:
# ここから実行フェーズ
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")


0 Batch accuracy: 0.1 Val accuracy: 0.1126
1 Batch accuracy: 0.04 Val accuracy: 0.1126
2 Batch accuracy: 0.08 Val accuracy: 0.1126
3 Batch accuracy: 0.06 Val accuracy: 0.1126
4 Batch accuracy: 0.1 Val accuracy: 0.1126
5 Batch accuracy: 0.1 Val accuracy: 0.1126
6 Batch accuracy: 0.14 Val accuracy: 0.1126
7 Batch accuracy: 0.16 Val accuracy: 0.1126
8 Batch accuracy: 0.04 Val accuracy: 0.1126
9 Batch accuracy: 0.1 Val accuracy: 0.1126
10 Batch accuracy: 0.14 Val accuracy: 0.1126
11 Batch accuracy: 0.08 Val accuracy: 0.1126
12 Batch accuracy: 0.06 Val accuracy: 0.1126
13 Batch accuracy: 0.08 Val accuracy: 0.1126
14 Batch accuracy: 0.22 Val accuracy: 0.1126
15 Batch accuracy: 0.12 Val accuracy: 0.1126
16 Batch accuracy: 0.1 Val accuracy: 0.1126
17 Batch accuracy: 0.1 Val accuracy: 0.1126
18 Batch accuracy: 0.08 Val accuracy: 0.1126
19 Batch accuracy: 0.1 Val accuracy: 0.1126
20 Batch accuracy: 0.06 Val accuracy: 0.1126
21 Batch accuracy: 0.06 Val accuracy: 0.1126
22 Batch accuracy: 0.06 Val

In [44]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)


INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [45]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
